In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

from data_loader import data_label_split
from data_loader import generate_data_set
from data_loader import dmso_taxol_ProfileBag
from torch_exp import mini_noise_signal_cv
from torch_exp import train

from model import SmallDeepSet

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold
from sklearn import datasets
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier

import torch
import torch.utils.data as D 
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim

In [44]:
drop_NA_data = pd.read_csv("moa_data_drop_NA.csv", index_col=0)
sf_drop_NA_data = drop_NA_data[["compound", "concentration",
                                "moa", "row ID", "Iteration (#2)", "COND",
                               "AreaShape_Area_Nuclei", "AreaShape_Compactness_Nuclei"]]

In [4]:
mini_data = pd.read_csv("mini_moa_data_drop_NA.csv",index_col=0)
# sf_drop_NA_data = mini_data[["compound", "concentration",
#                                 "moa", "row ID", "Iteration (#2)", "COND",
#                                "AreaShape_Area_Nuclei", "AreaShape_Compactness_Nuclei"]]

In [46]:

# bagData = dmso_taxol_ProfileBag(sf_drop_NA_data, 200, 50, 5, 0.5, "taxol", "DMSO", 0.5)
# dataloader = D.DataLoader(bagData, batch_size=1, shuffle=True)

In [42]:
# data = sf_drop_NA_data
# num_bag = 10
# bag_size_mean = 10
# bag_size_std = 0
# treatment = "taxol" 
# control = "DMSO"
# model = sf_deepset
# optimizer = optimizer
# splits = 4
# epochs = 1

# Deep Set Model

In [151]:
full_deepset = SmallDeepSet().cuda()

In [47]:
sf_deepset = SmallDeepSet().cuda()

In [48]:
optimizer = optim.Adam(sf_deepset.parameters(), lr=0.0005, betas=(0.9, 0.999), weight_decay=10e-5)

In [9]:
results = mini_noise_signal_cv(drop_NA_data, 1000, 100, 10, "taxol", "DMSO", full_deepset, 10, 50)

NameError: name 'drop_NA_data' is not defined

In [ ]:
sf_results = mini_noise_signal_cv(data = sf_drop_NA_data, 
                                  num_bag = 300,
                                  bag_size_mean = 100, 
                                  bag_size_std = 10, 
                                  treatment = "taxol", 
                                  control = "DMSO", 
                                  model = sf_deepset, 
                                  optimizer = optimizer,
                                  splits = 5,
                                  epochs = 2)

In [ ]:
sf_results

In [246]:
acc_control, acc_treat, pred_score_control, pred_score_treat = test(sf_deepset, dataloader)

In [29]:
(mean_control_accuracy, std_control_accuracy, 
            mean_treat_accuracy, std_treat_accuracy, 
            mean_pred_score_control, std_pred_score_control,
            mean_pred_score_treatment, std_pred_score_treatment) = sf_results

In [ ]:
dict = {'control_accuracy': nme, 'std_control_accuracy': deg, 'score': scr}  

In [24]:
sf_results = pd.DataFrame(list(sf_results)).transpose()

In [25]:
sf_results.columns = ["mean_control_accuracy", "std_control_accuracy", 
            "mean_treat_accuracy", "std_treat_accuracy", 
            "mean_pred_score_control", "std_pred_score_control",
            "mean_pred_score_treatment", "std_pred_score_treatment"]

In [26]:
sf_results

,mean_control_accuracy,std_control_accuracy,mean_treat_accuracy,std_treat_accuracy,mean_pred_score_control,std_pred_score_control,mean_pred_score_treatment,std_pred_score_treatment
0,0.6,0.489898,0.4,0.489898,0.392564,0.385677,0.393943,0.422387
1,0.1,0.300000,1.0,0.000000,0.722624,0.191698,0.814537,0.104709
2,0.4,0.489898,0.7,0.458258,0.543045,0.198318,0.673363,0.171835
3,0.9,0.300000,0.3,0.458258,0.213512,0.131968,0.393299,0.203889
4,0.9,0.300000,0.3,0.458258,0.187630,0.155302,0.295522,0.244676
5,0.7,0.458258,0.6,0.489898,0.324578,0.361401,0.679215,0.287370
6,0.8,0.400000,0.9,0.300000,0.343598,0.340132,0.728700,0.192789
7,1.0,0.000000,0.8,0.400000,0.134028,0.126734,0.733124,0.210020
8,0.6,0.489898,0.7,0.458258,0.386999,0.428574,0.691723,0.349524
9,0.7,0.458258,1.0,0.000000,0.397993,0.306416,0.989802,0.014643
